## <a id='1'>1. Import Libraries</a>

In [ ]:
!pip install selenium
!pip install tweepy
!pip install wordcloud
!pip install textblob

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import time
import string
import warnings

# for all NLP related operations on text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# To mock web-browser and scrap tweets
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# To consume Twitter's API
import tweepy
from tweepy import OAuthHandler 

# To identify the sentiment of text
from textblob import TextBlob

# ignoring all the warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# downloading stopwords corpus
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
stopwords = set(stopwords.words("english"))

# for showing all the plots inline
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Import Data

## <a id='3'>3. Text Pre-processing</a> 
### <a id='3A'>A. Pre-processing 'Key Words'</a>
#### <a id='3Aa'>a. Removing '@names'</a>
Here we can see that at many places we have '@names', which is of no use, since it don't have any meaning, So needs to be removed.

In [ ]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
        text = re.sub(r'#', '', text)
    return text

In [ ]:
# We are keeping cleaned tweets in a new column called 'tidy_tweets'
tweets_df['tidy_tweets'] = np.vectorize(remove_pattern)(tweets_df['tweet'], "@[\w]*: | *RT*")
tweets_df.head(10)

id  ...                                        tidy_tweets
0  1343849645521338369  ...                                                  💕
1  1343496082639077376  ...       @CheeseburgerOH Self-love at its finest... 🥰
2  1343472220341850112  ...              Monday mood.  https://t.co/WN7bEwbbBq
3  1343472023805198337  ...  @CheeseburgerOH @wEzfpAtbfO0qe7 What have I un...
4  1343151046483275777  ...  Crazy mofos is who I surround myself with.  We...
5  1343135985001873419  ...  Kinda upset I haven't seen Drunk Darren before...
6  1343119111912521729  ...  @SenyoritoJDL Sending you virtual hugs, brothe...
7  1343092205880180737  ...                                    Love you all. 🖤
8  1343085637893701634  ...         Fancy Darren is a Darren I've yet to meet.
9  1343083717632278528  ...  @wEzfpAtbfO0qe7 @AHOOMEE He drinks everyone's ...

[10 rows x 37 columns]

Seems good, now we don't have '@name' values

#### <a id='3Ab'>b. Removing links (http | https)</a>

In [ ]:
cleaned_tweets = []

for index, row in tweets_df.iterrows():
    # Here we are filtering out all the words that contains link
    words_without_links = [word for word in row.tidy_tweets.split() if 'http' not in word]
    cleaned_tweets.append(' '.join(words_without_links))

tweets_df['tidy_tweets'] = cleaned_tweets
tweets_df.head(10)

id  ...                                        tidy_tweets
0  1343849645521338369  ...                                                  💕
1  1343496082639077376  ...       @CheeseburgerOH Self-love at its finest... 🥰
2  1343472220341850112  ...                                       Monday mood.
3  1343472023805198337  ...  @CheeseburgerOH @wEzfpAtbfO0qe7 What have I un...
4  1343151046483275777  ...  Crazy mofos is who I surround myself with. Wei...
5  1343135985001873419  ...  Kinda upset I haven't seen Drunk Darren before...
6  1343119111912521729  ...  @SenyoritoJDL Sending you virtual hugs, brother 🖤
7  1343092205880180737  ...                                    Love you all. 🖤
8  1343085637893701634  ...         Fancy Darren is a Darren I've yet to meet.
9  1343083717632278528  ...  @wEzfpAtbfO0qe7 @AHOOMEE He drinks everyone's ...

[10 rows x 37 columns]

#### <a id='3Ac'>c. Removing tweets with empty text</a>

In [ ]:
tweets_df = tweets_df[tweets_df['tidy_tweets']!='']
tweets_df.head()

id  ...                                        tidy_tweets
0  1343849645521338369  ...                                                  💕
1  1343496082639077376  ...       @CheeseburgerOH Self-love at its finest... 🥰
2  1343472220341850112  ...                                       Monday mood.
3  1343472023805198337  ...  @CheeseburgerOH @wEzfpAtbfO0qe7 What have I un...
4  1343151046483275777  ...  Crazy mofos is who I surround myself with. Wei...

[5 rows x 37 columns]

#### <a id='3Ad'>d. Dropping duplicate rows</a>

In [ ]:
tweets_df.drop_duplicates(subset=['tidy_tweets'], keep=False)
tweets_df.head()

id  ...                                        tidy_tweets
0  1343849645521338369  ...                                                  💕
1  1343496082639077376  ...       @CheeseburgerOH Self-love at its finest... 🥰
2  1343472220341850112  ...                                       Monday mood.
3  1343472023805198337  ...  @CheeseburgerOH @wEzfpAtbfO0qe7 What have I un...
4  1343151046483275777  ...  Crazy mofos is who I surround myself with. Wei...

[5 rows x 37 columns]

#### <a id='3Ae'>e. Resetting index</a>
It seems that our index needs to be reset, since after removal of some rows, some index values are missing, which may cause problem in future operations.

In [ ]:
tweets_df = tweets_df.reset_index(drop=True)
tweets_df.head()

id  ...                                        tidy_tweets
0  1343849645521338369  ...                                                  💕
1  1343496082639077376  ...       @CheeseburgerOH Self-love at its finest... 🥰
2  1343472220341850112  ...                                       Monday mood.
3  1343472023805198337  ...  @CheeseburgerOH @wEzfpAtbfO0qe7 What have I un...
4  1343151046483275777  ...  Crazy mofos is who I surround myself with. Wei...

[5 rows x 37 columns]

#### <a id='3Af'>f. Removing Punctuations, Numbers and Special characters</a>
This step should not be followed if we also want to do sentiment analysis on __key phrases__ as well, because semantic meaning in a sentence needs to be present. So here we will create one additional column 'absolute_tidy_tweets' which will contain absolute tidy words which can be further used for sentiment analysis on __key words__.

In [ ]:
tweets_df['absolute_tidy_tweets'] = tweets_df['tidy_tweets'].str.replace("[^a-zA-Z# ]", "")

#### <a id='3Ag'>g. Removing Stop words</a>
With the same reason we mentioned above, we won't perform this on 'tidy_tweets' column, because it needs to be used for __key_phrases__ sentiment analysis.

In [ ]:
stopwords_set = set(stopwords)
cleaned_tweets = []

for index, row in tweets_df.iterrows():
    
    # filerting out all the stopwords 
    words_without_stopwords = [word for word in row.absolute_tidy_tweets.split() if not word in stopwords_set]
    
    # finally creating tweets list of tuples containing stopwords(list) and sentimentType 
    cleaned_tweets.append(' '.join(words_without_stopwords))

tweets_df['absolute_tidy_tweets'] = cleaned_tweets
tweets_df.head(10)

id  ...                               absolute_tidy_tweets
0  1343849645521338369  ...                                                   
1  1343496082639077376  ...                     CheeseburgerOH Selflove finest
2  1343472220341850112  ...                                        Monday mood
3  1343472023805198337  ...       CheeseburgerOH wEzfpAtbfOqe What I unleashed
4  1343151046483275777  ...  Crazy mofos I surround Weird ppl weird ppl lif...
5  1343135985001873419  ...  Kinda upset I havent seen Drunk Darren While h...
6  1343119111912521729  ...          SenyoritoJDL Sending virtual hugs brother
7  1343092205880180737  ...                                               Love
8  1343085637893701634  ...                   Fancy Darren Darren Ive yet meet
9  1343083717632278528  ...   wEzfpAtbfOqe AHOOMEE He drinks everyones portion

[10 rows x 38 columns]

#### <a id='3Ah'>h. Tokenize *'absolute_tidy_tweets'*</a>  

In [ ]:
tokenized_tweet = tweets_df['absolute_tidy_tweets'].apply(lambda x: x.split())
tokenized_tweet.head()

0                                                   []
1                   [CheeseburgerOH, Selflove, finest]
2                                       [Monday, mood]
3    [CheeseburgerOH, wEzfpAtbfOqe, What, I, unleas...
4    [Crazy, mofos, I, surround, Weird, ppl, weird,...
Name: absolute_tidy_tweets, dtype: object

#### <a id='3Ai'>i. Converting words to Lemma</a>

In [ ]:
word_lemmatizer = WordNetLemmatizer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [word_lemmatizer.lemmatize(i) for i in x])
tokenized_tweet.head()

0                                                   []
1                   [CheeseburgerOH, Selflove, finest]
2                                       [Monday, mood]
3    [CheeseburgerOH, wEzfpAtbfOqe, What, I, unleas...
4    [Crazy, mofos, I, surround, Weird, ppl, weird,...
Name: absolute_tidy_tweets, dtype: object

#### <a id='3Aj'>j. Joining all tokens into sentences</a>

In [ ]:
for i, tokens in enumerate(tokenized_tweet):
    tokenized_tweet[i] = ' '.join(tokens)

tweets_df['absolute_tidy_tweets'] = tokenized_tweet
tweets_df.head(10)

id  ...                               absolute_tidy_tweets
0  1343849645521338369  ...                                                   
1  1343496082639077376  ...                     CheeseburgerOH Selflove finest
2  1343472220341850112  ...                                        Monday mood
3  1343472023805198337  ...       CheeseburgerOH wEzfpAtbfOqe What I unleashed
4  1343151046483275777  ...  Crazy mofos I surround Weird ppl weird ppl lif...
5  1343135985001873419  ...  Kinda upset I havent seen Drunk Darren While h...
6  1343119111912521729  ...           SenyoritoJDL Sending virtual hug brother
7  1343092205880180737  ...                                               Love
8  1343085637893701634  ...                   Fancy Darren Darren Ive yet meet
9  1343083717632278528  ...    wEzfpAtbfOqe AHOOMEE He drink everyones portion

[10 rows x 38 columns]

### <a id='3B'>B. Pre-processing 'Key Phrases'</a> 

#### <a id='3Ba'>a. Helper class, will help in preprocessing phrase terms</a>

In [ ]:
class PhraseExtractHelper(object):
    def __init__(self):
        self.lemmatizer = nltk.WordNetLemmatizer()
        self.stemmer = nltk.stem.porter.PorterStemmer()
    
    def leaves(self, tree):
        """Finds NP (nounphrase) leaf nodes of a chunk tree."""
        for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
            yield subtree.leaves()

    def normalise(self, word):
        """Normalises words to lowercase and stems and lemmatizes it."""
        word = word.lower()
        # word = self.stemmer.stem_word(word) # We will loose the exact meaning of the word 
        word = self.lemmatizer.lemmatize(word)
        return word

    def acceptable_word(self, word):
        """Checks conditions for acceptable word: length, stopword. We can increase the length if we want to consider large phrase"""
        accepted = bool(3 <= len(word) <= 40
            and word.lower() not in stopwords
            and 'https' not in word.lower()
            and 'http' not in word.lower()
            and '#' not in word.lower()
            )
        return accepted

    def get_terms(self, tree):
        for leaf in self.leaves(tree):
            term = [ self.normalise(w) for w,t in leaf if self.acceptable_word(w) ]
            yield term

#### <a id='3Bb'>b. Grammatical rule to identify phrases</a>

In [ ]:
sentence_re = r'(?:(?:[A-Z])(?:.[A-Z])+.?)|(?:\w+(?:-\w+)*)|(?:\$?\d+(?:.\d+)?%?)|(?:...|)(?:[][.,;"\'?():-_`])'
grammar = r"""
    NBAR:
        {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
"""
chunker = nltk.RegexpParser(grammar)

#### <a id='3Bc'>c. New feature called 'key_phrases', will contain phrases for corresponding tweet</a>

In [ ]:
key_phrases = []
phrase_extract_helper = PhraseExtractHelper()

for index, row in tweets_df.iterrows(): 
    toks = nltk.regexp_tokenize(row.tidy_tweets, sentence_re)
    postoks = nltk.tag.pos_tag(toks)
    tree = chunker.parse(postoks)

    terms = phrase_extract_helper.get_terms(tree)
    tweet_phrases = []

    for term in terms:
        if len(term):
            tweet_phrases.append(' '.join(term))
    
    key_phrases.append(tweet_phrases)
    
tweets_df['key_phrases'] = key_phrases
tweets_df.head()

id  ...                                        key_phrases
0  1343849645521338369  ...                                                 []
1  1343496082639077376  ...                         [cheeseburgeroh self-love]
2  1343472220341850112  ...                                      [monday mood]
3  1343472023805198337  ...                              [cheeseburgeroh  @we]
4  1343151046483275777  ...  [crazy mofos, weird ppl, weird ppl life, regre...

[5 rows x 39 columns]

# Analysis

In [ ]:
!pip install --upgrade orjson

In [ ]:
def parse_files():
    import orjson
    
    columns = ['id','conversation_id','created_at','date','time','timezone','userid','username','name','place','tweet','language','mentions','urls','photos','replies_count','retweets_count','likes_count','hashtags','cashtags','link','retweet','quote_url','video','thumbnail','near','geo','source','user_rt_id','user_rt','retweet_id','reply_to','retweet_data','translate','trans_src','trans_dest','tidy_tweets','absolute_tidy_tweets','key-phrases','textblob score','vader score']
    dataframe = pd.DataFrame(columns=[])
    
    for file in glob.glob('*.xlsx'):
        tweets=pd.read_excel(file)
        dataframe = dataframe.append(pd.DataFrame(tweets))
    return dataframe

In [ ]:
#Upload all the individual excel files to the file tab of the Google Colab
df_final = parse_files()

In [ ]:
df_final.shape

In [ ]:
from datetime import datetime
import pandas as pd

def date_function(text):
    return text[:4]

def real_date(date):
    my_date=datetime_object =datetime.strptime(date, '%Y').date()
    return my_date

df['Date'] = df['created_at'].apply(date_function)


pd.set_option('display.max_columns', None)
df

import text2emotion as te
def emotion_function(text):
    return te.get_emotion(text)


#df['Tweets_Emotion'] = df['YOUR_TWEETS_COLUMN'].apply(emotion_function)
#a=df['absolute_tidy_tweets'].astype(str).apply(emotion_function)
df['Tweets_Emotion']=a
df.info()



import pandas as pd
df2 = pd.json_normalize(df['Tweets_Emotion'])

#df.index = pd.to_datetime(df['date'],format='%y-%m')
#df.groupby(by=[df.index.year])

df2.tail()
new_df=df2.join(df)
new_df


import numpy as np
from matplotlib import pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
from scipy.interpolate import interp1d
import scipy as sp
!pip install vincent
import vincent
import pandas as pd
import random
import matplotlib.dates as mdates



#import plotly.express as px
#fig = px.line(data)
#fig.show()

fig, ax = plt.subplots(figsize=(15,7))
new_df.groupby(['Date']).sum()['Happy'].plot(ax=ax,legend="Happy")
new_df.groupby(['Date']).sum()['Sad'].plot(ax=ax,legend="Sad")
new_df.groupby(['Date']).sum()['Fear'].plot(ax=ax,legend="Fear")
new_df.groupby(['Date']).sum()['Surprise'].plot(ax=ax,legend="Surprise")
new_df.groupby(['Date']).sum()['Angry'].plot(ax=ax,legend="Angry")
x_smooth.plot(ax=ax

new_df